In [1]:
import os 
import sys
import pandas as pd
import numpy as np
import operator
import random
from functools import reduce

from bson import ObjectId

sys.path.append(os.path.abspath(os.path.join('..', '..')))

from services.pandas.pandas_service import PandasService
from services.database.database_service import Environment

In [22]:
env = Environment().get_instance()
pd_services = PandasService()
db_daily = env.get_db().get('daily')
db_users = env.get_db().get('users')

2024-05-24 17:24:27,301 INFO root : Environment (__init__): making connection
2024-05-24 17:24:27,308 INFO root : Environment (get_db): access database
2024-05-24 17:24:27,312 INFO root : Environment (get_db): access database


In [25]:
wh_collection = db_daily.get_collection('WhatHappenCollection')
fs_collection = db_daily.get_collection('SentimentCollection')
ts_collection = db_daily.get_collection('TrashHappenCollection')
fl_collection = db_users.get_collection('FollowsCollection')
cs_collection = db_daily.get_collection('CommentsCollection')

In [6]:
list(wh_collection.aggregate([
    {'$match': {'user_id': ObjectId("65ae90883470eb010f5f1104")}},
    {'$group': {'_id': None, 'total': {'$sum': 1 } } },
    {'$project': { '_id': 0 } }
] ))


[{'total': 22}]

In [11]:
list(fs_collection.aggregate([
    {'$match': {'user_id': ObjectId('6644c4eeff47ba18dd110294')}},
]))

[{'_id': ObjectId('664fb194766e30ff52bf61cb'),
  'created_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 896000),
  'updated_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 896000),
  'text': 'teste edite and create',
  'sentiment': 'negative',
  'name_model': 'logistic_regression',
  'model_version': '1.0.2',
  'accuracy_score': 0.8870849160642078,
  'score_positive': 0.4873570721523508,
  'score_negative': 0.5126429278476492,
  'user_id': ObjectId('6644c4eeff47ba18dd110294'),
  'what_happen_id': ObjectId('664fb194766e30ff52bf61ca')}]

In [20]:
list(ts_collection.find())

[{'_id': ObjectId('6650f24d3f010385da26228c'),
  'created_at': datetime.datetime(2024, 5, 24, 17, 2, 21, 223000),
  'updated_at': datetime.datetime(2024, 5, 24, 17, 2, 21, 223000),
  'user_id': ObjectId('661f2d5e30ee4c696a1d15fd'),
  'happen_id': ObjectId('664fb432766e30ff52bf6310'),
  'owner_happen_id': ObjectId('663ed74204ba683d9499b0a6')}]

In [19]:
matches = {'user_id': ObjectId('661f2d5e30ee4c696a1d15fd')}
trash_lookup = {'as': 'trash', 'from': 'TrashHappenCollection', 'localField': '_id', 'foreignField': 'happen_id'}
feeling_lookup = {'as': 'feelings', 'from': 'SentimentCollection', 'localField': '_id', 'foreignField': 'what_happen_id'}

list(wh_collection.aggregate([
    {'$match': matches},
    {'$lookup': trash_lookup},
    {'$lookup': feeling_lookup}
]))

[{'_id': ObjectId('664ce32d089c4c2e6bd9589c'),
  'created_at': datetime.datetime(2024, 5, 24, 15, 49, 57, 836000),
  'updated_at': datetime.datetime(2024, 5, 24, 15, 49, 57, 836000),
  'what_happen': 'Tive um ótimo dia! que pena, apesar de tudo ter dado certo, nem tudo é o que parece. No entando acredito eu que no final do dia tudo pode se resolve.\n\nPreciso saber que um dia tudo vai ficar bem - mesmo sabendo que as pessas são ruins por natureza. Sem pensar nas coisas simplesmente acontece quando tem que acontecer - isso é verdade, pois eu acredito em Deus no final das contas.\n\nUm dia tudo vai se resolver!',
  'user_id': ObjectId('661f2d5e30ee4c696a1d15fd'),
  'name_id': 'root@daily',
  'visibility': 'PRIVATE',
  'likes': None,
  'feelings': [{'_id': ObjectId('664ce32d089c4c2e6bd9589d'),
    'created_at': datetime.datetime(2024, 5, 24, 15, 49, 57, 850000),
    'updated_at': datetime.datetime(2024, 5, 24, 15, 49, 57, 850000),
    'text': 'Tive um ótimo dia! que pena, apesar de tudo t

In [28]:
list(wh_collection.find({'_id': ObjectId('661f2d5e30ee4c696a1d15fd')}))

[]

In [29]:
list(fs_collection.find({'what_happen_id': '661f2d5e30ee4c696a1d15fd'}))

[]

In [37]:
list(ts_collection.find())

[{'_id': ObjectId('6650f24d3f010385da26228c'),
  'created_at': datetime.datetime(2024, 5, 24, 17, 2, 21, 223000),
  'updated_at': datetime.datetime(2024, 5, 24, 17, 2, 21, 223000),
  'user_id': ObjectId('661f2d5e30ee4c696a1d15fd'),
  'happen_id': ObjectId('664fb432766e30ff52bf6310'),
  'owner_happen_id': ObjectId('663ed74204ba683d9499b0a6')}]

In [168]:
results = list(map(lambda x: x.get('following_id'),
                        list(fl_collection.find({'user_id': ObjectId('661f2d5e30ee4c696a1d15fd'), 'status': {'$eq': 'FOLLOWING'}}))))

trash_results = list(map(lambda x: x.get('happen_id'), list(ts_collection.find({'user_id': ObjectId('661f2d5e30ee4c696a1d15fd')}))))

ids = [ObjectId('6648b90dd13f9dcdb632f759')] + results

def query(id):
        trash_lookup = {'as': 'trash', 'from': 'TrashHappenCollection', 'localField': '_id', 'foreignField': 'happen_id'}
        lookup = {'as': 'feelings', 'from': 'SentimentCollection', 'localField': '_id', 'foreignField': 'what_happen_id'}
        return [
                {'$match': {'user_id': id}},
                {'$lookup': trash_lookup},
                {'$lookup': lookup},
                {'$match': {'_id': {'$nin': trash_results} }},
                {'$unset': ['trash']}
                ]

results = reduce(operator.concat, [list(wh_collection.aggregate(query(id))) for id in ids ])

results

[{'_id': ObjectId('664fb194766e30ff52bf61ca'),
  'created_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 875000),
  'updated_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 875000),
  'what_happen': 'teste edite and create',
  'user_id': ObjectId('6644c4eeff47ba18dd110294'),
  'name_id': 'joao@daily',
  'visibility': 'PUBLIC',
  'likes': None,
  'feelings': [{'_id': ObjectId('664fb194766e30ff52bf61cb'),
    'created_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 896000),
    'updated_at': datetime.datetime(2024, 5, 23, 18, 13, 56, 896000),
    'text': 'teste edite and create',
    'sentiment': 'negative',
    'name_model': 'logistic_regression',
    'model_version': '1.0.2',
    'accuracy_score': 0.8870849160642078,
    'score_positive': 0.4873570721523508,
    'score_negative': 0.5126429278476492,
    'user_id': ObjectId('6644c4eeff47ba18dd110294'),
    'what_happen_id': ObjectId('664fb194766e30ff52bf61ca')}]}]

In [ ]:
    # for doc in self.collection.find():
    #     doc['name_id'] = "beto@daily.com"
    #     # doc['name'] = 'beto'
    #     doc['likes'] = None
    #     doc['visibility'] = 'PRIVATE'
    #     doc['updated_at'] = datetime.datetime.now()
    #     self.collection.replace_one({Var.ID.value: doc[Var.ID.value]}, doc)

In [44]:
# for doc in wh_collection.find():
#     doc['user_id'] = ObjectId(doc['user_id'])
#     wh_collection.replace_one({'_id':doc['_id']}, doc)

# for doc in fs_collection.find():
#     doc['user_id'] = ObjectId(doc['user_id'])
#     doc['what_happen_id'] = ObjectId(doc['what_happen_id'])
#     fs_collection.replace_one({'_id': doc['_id']}, doc)